In [ ]:
# Dependency installation code
# !pip install PyPDF2 torch transformers

# Import dependencies
import PyPDF2
import torch
from transformers import AutoTokenizer, T5EncoderModel

# Load T5EncoderModel
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = T5EncoderModel.from_pretrained("google-t5/t5-small")

In [27]:
# Tokenize Input sentence and query
input_ids = tokenizer('Our product is sustainable', return_tensors = 'pt').input_ids
query_ids = tokenizer('Sustainable ', return_tensors = 'pt').input_ids

# Generate Encodings
outputs = model(input_ids=input_ids)
query = model(input_ids = query_ids)

# Retrieve encodings for sentence and query.
last_hidden_states = outputs.last_hidden_state
last_hidden_states_query = query.last_hidden_state


In [28]:
# Apply max pooling along the token dimension (dim=1)
pooled_last_hidden_states = torch.max(last_hidden_states, dim=1).values.unsqueeze(0)  # Add batch dimension
pooled_last_hidden_states_query = torch.max(last_hidden_states_query, dim=1).values.unsqueeze(0)  # Add batch dimension

# Reshape pooled_last_hidden_states_query to match the shape of pooled_last_hidden_states
pooled_last_hidden_states_query = pooled_last_hidden_states_query.view(1, -1)

# Compute dot product between the pooled representations
similarity = torch.matmul(pooled_last_hidden_states, pooled_last_hidden_states_query.T)

# Print similarity score
print("Similarity Score:", similarity.item())


Similarity Score: 9.385696411132812


Current Results:

| Sentence                     | Query                        | Score |
|------------------------------|------------------------------|-------|
| 'This product is sustainable'| 'Where is the India? '       | 14.40 |
| 'India is in Asia'           | 'Where is the India? '       | 15.46 |
| 'Our product is sustainable' | 'Sustainable '               | 09.38 |
